**Table of contents**<a id='toc0_'></a>    
- [<b>Import Libraries</b>](#toc1_)    
- [<b>1. Load Dataset</b>](#toc2_)    
  - [Load Configuration File](#toc2_1_)    
  - [Data Collection](#toc2_2_)    
- [<b>2. Data Validation </b>](#toc3_)    
  - [Types of Data](#toc3_1_)    
  - [Data Range](#toc3_2_)    
  - [<b>Data Dimension </b>](#toc3_3_)    
- [<b>Handling Columns Type</b>](#toc4_)    
  - [Check Dataset Type](#toc4_1_)    
- [<b>Handling data category</b>](#toc5_)    
- [<b>Data Defense</b>](#toc6_)    
- [<b>Splitting Data</b>](#toc7_)    
  - [<b>Split Train and Test Data (7:3)</b>](#toc7_1_)    
  - [<b>Split Valid and Test Data (1:1)</b>](#toc7_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[<b>Import Libraries</b>](#toc0_)

In [1]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import src.util as util
import joblib
import os
import yaml
import copy

import warnings
warnings.filterwarnings('ignore')

In [2]:
class clr:
    start = '\033[93m' + '\033[1m'
    bold = '\033[1m'
    underline = '\033[4m'
    color = '\033[93m'
    end = '\033[0m'

# <a id='toc2_'></a>[<b>1. Load Dataset</b>](#toc0_)

## <a id='toc2_1_'></a>[Load Configuration File](#toc0_)

In [3]:
config_data = util.load_config()

## <a id='toc2_2_'></a>[Data Collection](#toc0_)

In [4]:
def read_raw_data(config: dict) -> pd.DataFrame:
    # Create variable to store raw dataset
    raw_dataset = pd.DataFrame()

    # Raw Dataset Dir
    raw_dataset_dir = config["raw_dataset_dir"]

    # Look and load add csv files
    for i in tqdm(os.listdir(raw_dataset_dir)):
        raw_dataset = pd.concat([pd.read_csv(raw_dataset_dir + i), raw_dataset])
    
    # Return raw dataset
    return raw_dataset

In [5]:
raw_dataset = read_raw_data(config_data)

100%|██████████| 1/1 [00:00<00:00, 79.64it/s]


In [6]:
# dataset = pd.read_csv('insurance_claims.csv')
pd.set_option('display.max_columns', None)

raw_dataset.head().style.background_gradient(cmap='YlOrBr')

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.910000,0,466132,MALE,MD,craft-repair,sleeping,husband,53300,0,2015-01-25,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,9935 4th Drive,5,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y,nan
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.220000,5000000,468176,MALE,MD,machine-op-inspct,reading,other-relative,0,0,2015-01-21,Vehicle Theft,?,Minor Damage,Police,VA,Riverwood,6608 MLK Hwy,8,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2007,Y,nan
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.140000,5000000,430632,FEMALE,PhD,sales,board-games,own-child,35100,0,2015-02-22,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,7121 Francis Lane,7,3,NO,2,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,nan
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.740000,6000000,608117,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,2015-01-10,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,6956 Maple Drive,5,1,?,1,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,nan
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.910000,6000000,610706,MALE,Associate,sales,board-games,unmarried,66000,-46000,2015-02-17,Vehicle Theft,?,Minor Damage,None,NY,Arlington,3041 3rd Ave,20,1,NO,0,1,NO,6500,1300,650,4550,Accura,RSX,2009,N,nan


In [7]:
# Save raw dataset to file
util.pickle_dump(raw_dataset, config_data["raw_dataset_path"])

# <a id='toc3_'></a>[<b>2. Data Validation </b>](#toc0_)

## <a id='toc3_1_'></a>[Types of Data](#toc0_)

In [8]:
raw_dataset.dtypes

months_as_customer               int64
age                              int64
policy_number                    int64
policy_bind_date                object
policy_state                    object
policy_csl                      object
policy_deductable                int64
policy_annual_premium          float64
umbrella_limit                   int64
insured_zip                      int64
insured_sex                     object
insured_education_level         object
insured_occupation              object
insured_hobbies                 object
insured_relationship            object
capital-gains                    int64
capital-loss                     int64
incident_date                   object
incident_type                   object
collision_type                  object
incident_severity               object
authorities_contacted           object
incident_state                  object
incident_city                   object
incident_location               object
incident_hour_of_the_day 

## <a id='toc3_2_'></a>[Data Range](#toc0_)

In [9]:
# some columns have minimum value of zero, this value has been encoded as a missing value 
# instead of None Value or Misig cell.

raw_dataset.describe().style.set_sticky(axis="index").background_gradient()

,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year,_c39
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,0.000000
mean,203.954000,38.948000,546238.648000,1136.000000,1256.406150,1101000.000000,501214.488000,25126.100000,-26793.700000,11.644000,1.839000,0.992000,1.487000,52761.940000,7433.420000,7399.570000,37928.950000,2005.103000,nan
std,115.113174,9.140287,257063.005276,611.864673,244.167395,2297406.598118,71701.610941,27872.187708,28104.096686,6.951373,1.018880,0.820127,1.111335,26401.533190,4880.951853,4824.726179,18886.252893,6.015861,nan
min,0.000000,19.000000,100804.000000,500.000000,433.330000,-1000000.000000,430104.000000,0.000000,-111100.000000,0.000000,1.000000,0.000000,0.000000,100.000000,0.000000,0.000000,70.000000,1995.000000,nan
25%,115.750000,32.000000,335980.250000,500.000000,1089.607500,0.000000,448404.500000,0.000000,-51500.000000,6.000000,1.000000,0.000000,1.000000,41812.500000,4295.000000,4445.000000,30292.500000,2000.000000,nan
50%,199.500000,38.000000,533135.000000,1000.000000,1257.200000,0.000000,466445.500000,0.000000,-23250.000000,12.000000,1.000000,1.000000,1.000000,58055.000000,6775.000000,6750.000000,42100.000000,2005.000000,nan
75%,276.250000,44.000000,759099.750000,2000.000000,1415.695000,0.000000,603251.000000,51025.000000,0.000000,17.000000,3.000000,2.000000,2.000000,70592.500000,11305.000000,10885.000000,50822.500000,2010.000000,nan
max,479.000000,64.000000,999435.000000,2000.000000,2047.590000,10000000.000000,620962.000000,100500.000000,0.000000,23.000000,4.000000,2.000000,3.000000,114920.000000,21450.000000,23670.000000,79560.000000,2015.000000,nan


## <a id='toc3_3_'></a>[<b>Data Dimension </b>](#toc0_)

In [10]:
raw_dataset.shape

(1000, 40)

# <a id='toc4_'></a>[<b>Handling Columns Type</b>](#toc0_)

Change <b>Capital-gains</b> and <b>capital-loss</b> column name

In [11]:
raw_dataset =raw_dataset.rename(columns = {'capital-gains':'capital_gains'})
raw_dataset =raw_dataset.rename(columns = {'capital-loss':'capital_loss'})


Convert dataset <i>datetime</i>, <br>
convert data type <b>float</b> into <b>int32</b>, <br> 
convert data type <b>int64</b> into <b>int32</b> and <br>
convert few of data type <b>int</b> into <b>object</b> 

In [12]:
def type_data(set_data):
    """Change raw dataset type.

    Convert raw dataset datetime, float into int, int64 to int32 and convert int columns into object
    """
    # --- Change datetime object ---#
    for col in config_data["datetime_columns"]:
        set_data[col] = pd.to_datetime(set_data[col])

    # --- Columns _c39 is not defined columns, then drop the columns -- #
    set_data = set_data.drop(columns = ['_c39'], axis=1)

    # --- change columns type policy_annual_premium columns into int64 ---#
    set_data = set_data.astype({col: 'int32' for col in set_data.select_dtypes('float64').columns})

    # --- convert int64 into int32 --- #
    set_data = set_data.astype({col: 'int32' for col in set_data.select_dtypes('int64').columns})

    # --- convert few of int columns into object --- #
    raw_dataset_date = set_data[config_data['datetime_columns']]
    raw_dataset_num = set_data[config_data['int32_col']]

    list_of_non_cat = config_data['int32_col'].copy()
    list_of_non_cat = list_of_non_cat + config_data['datetime_columns']

    raw_dataset_cat = set_data[config_data['object_columns']]
    raw_dataset_cat = raw_dataset_cat.astype(str)

    # --- Return raw dataset with datetime, numerical int, and category object --#
    raw_dataset = pd.concat([raw_dataset_date, raw_dataset_num, raw_dataset_cat], axis = 1)
    
    # --- Handle umbrella limit data --- #
    raw_dataset = raw_dataset.replace({'umbrella_limit': '-1000000'}, '1000000') 

    return raw_dataset

In [13]:
raw_dataset = type_data(raw_dataset)

## <a id='toc4_1_'></a>[Check Dataset Type](#toc0_)

In [14]:
## Check dataset type of each features
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   policy_bind_date             1000 non-null   datetime64[ns]
 1   incident_date                1000 non-null   datetime64[ns]
 2   months_as_customer           1000 non-null   int32         
 3   age                          1000 non-null   int32         
 4   policy_number                1000 non-null   int32         
 5   policy_annual_premium        1000 non-null   int32         
 6   insured_zip                  1000 non-null   int32         
 7   capital_gains                1000 non-null   int32         
 8   capital_loss                 1000 non-null   int32         
 9   incident_hour_of_the_day     1000 non-null   int32         
 10  total_claim_amount           1000 non-null   int32         
 11  injury_claim                 1000 non-null  

In [15]:
## Sanity Check
raw_dataset.shape

(1000, 35)

# <a id='toc5_'></a>[<b>Handling data category</b>](#toc0_)

In [27]:
## Check categorical data
pd.set_option('display.max_rows', None)

pd.DataFrame(
        raw_dataset[raw_dataset.select_dtypes('object').columns.to_list()]
        .melt(var_name='columns', value_name='index')\
        .value_counts())\
        .sort_values(by=['columns',0])



0
columns                     index                        
authorities_contacted       None                       91
                            Ambulance                 196
                            Other                     198
                            Fire                      223
                            Police                    292
auto_make                   Honda                      55
                            Mercedes                   65
                            Jeep                       67
                            Accura                     68
                            Volkswagen                 68
                            Audi                       69
                            Toyota                     70
                            Ford                       72
                            BMW                        72
                            Chevrolet                  76
                            Nissan                     78
                            Dodge                      80
                            Suburu                     80
                            Saab                       80
auto_model                  RSX                        12
                            Accord                     13
                            M5                         15
                            X6                         16
                            3 Series                   18
                            C300                       18
                            Impreza                    20
                            CRV                        20
                            TL                         20
                            ML350                      20
                            Corolla                    20
                            Fusion                     21
                            Silverado                  22
                            Highlander                 22
                            Civic                      22
                            X5                         23
                            Ultima                     23
                            Maxima                     24
                            Escape                     24
                            Tahoe                      24
                            93                         25
                            Grand Cherokee             25
                            95                         27
                            E400                       27
                            F150                       27
                            Forrestor                  28
                            92x                        28
                            Camry                      28
                            Malibu                     30
                            Pathfinder                 31
                            A5                         32
                            Legacy                     32
                            Passat                     33
                            Jetta                      35
                            MDX                        36
                            A3                         37
                            Neon                       37
                            Wrangler                   42
                            RAM                        43
auto_year                   1996                       37
                            2004                       39
                            1998                       40
                            2001                       42
                            2000                       42
                            2014                       44
                            2008                       45
                            1997                       46
                            2012                       46
                            2015                      

We can set to drop <b>incident_location</b>, since it has <b>unique values.</b>

In [32]:
# --- drop incident_location --- #
# raw_dataset = raw_dataset.drop(columns='incident_location')

There's value of <b>'?' and 'nan'</b> in several features, we can't drop this value <br>
since it will significantly drop the instances of dataset, hence we convert <b>'?'</b> and <b>'nan'</b> into <b>'Unknown'</b> <br>
We will reconsider to fill this unknown value on <b>imputation step</b>

In [39]:
# --- Change ? category value into UNKNOWN

raw_dataset.collision_type = raw_dataset.collision_type.replace("?","UNKNOWN")
raw_dataset.property_damage = raw_dataset.property_damage.replace("?", "UNKNOWN")
raw_dataset.police_report_available = raw_dataset.police_report_available.replace("?", "UNKNOWN")
raw_dataset.authorities_contacted = raw_dataset.authorities_contacted.replace("None", "UNKNOWN")

In [34]:
# Check label data
raw_dataset.fraud_reported.value_counts(normalize=True)

N    0.753
Y    0.247
Name: fraud_reported, dtype: float64

# <a id='toc6_'></a>[<b>Data Defense</b>](#toc0_)

In [35]:
def check_data(input_data, params, api = False):
    input_data = copy.deepcopy(input_data)
    params = copy.deepcopy(params)

    if not api:
        # Check data types
        assert input_data.select_dtypes("datetime").columns.to_list() == \
            params["datetime_columns"], "an error occurs in datetime column(s)."
        assert input_data.select_dtypes("object").columns.to_list() == \
            params["object_columns"], "an error occurs in object column(s)."
        assert input_data.select_dtypes("int").columns.to_list() == \
            params["int32_col"], "an error occurs in int32 column(s)."
    else:
        # In case checking data from api
        # Predictor that has object dtype only stasiun
        object_columns = params["object_columns"]
        del object_columns[1:]

        # Max column not used as predictor
        int_columns = params["int32_col"]
        del int_columns[-1]

        # Check data types
        assert input_data.select_dtypes("object").columns.to_list() == \
            object_columns, "an error occurs in object column(s)."
        assert input_data.select_dtypes("int").columns.to_list() == \
            int_columns, "an error occurs in int32 column(s)."
        
    # Check data ranges
    assert set(input_data.policy_deductable).issubset(set(params["policy_deductable"])), \
        "an error occurs in policy_deductable range."
    assert set(input_data.umbrella_limit).issubset(set(params["umbrella_limit"])), \
        "an error occurs in umbrella_limit range."
    assert set(input_data.number_of_vehicles_involved).issubset(set(params["number_of_vehicles_involved"])), \
        "an error occurs in number_of_vehicles_involved range."
    assert set(input_data.bodily_injuries).issubset(set(params["bodily_injuries"])), \
        "an error occurs in bodily_injuries range."
    assert set(input_data.witnesses).issubset(set(params["witnesses"])), \
        "an error occurs in witnesses range."
    assert set(input_data.auto_year).issubset(set(params["auto_year"])), \
        "an error occurs in auto_year range."
    assert set(input_data.policy_state).issubset(set(params["policy_state"])), \
        "an error occurs in policy_state range."
    assert set(input_data.policy_csl).issubset(set(params["policy_csl"])), \
        "an error occurs in policy_csl range."
    assert set(input_data.insured_sex).issubset(set(params["insured_sex"])), \
        "an error occurs in insured_sex range."
    assert set(input_data.insured_hobbies).issubset(set(params["insured_hobbies"])), \
        "an error occurs in insured_hobbies range."
    assert set(input_data.incident_type).issubset(set(params["incident_type"])), \
        "an error occurs in incident_type range."
    assert set(input_data.collision_type).issubset(set(params["collision_type"])), \
        "an error occurs in collision_type range."
    assert set(input_data.incident_severity).issubset(set(params["incident_severity"])), \
        "an error occurs in incident_severity range."
    assert set(input_data.authorities_contacted).issubset(set(params["authorities_contacted"])), \
        "an error occurs in authorities_contacted range."
    assert set(input_data.incident_state).issubset(set(params["incident_state"])), \
        "an error occurs in incident_state range."
    assert set(input_data.incident_city).issubset(set(params["incident_city"])), \
        "an error occurs in incident_city range."
    assert set(input_data.property_damage).issubset(set(params["property_damage"])), \
        "an error occurs in property_damage range."
    assert set(input_data.police_report_available).issubset(set(params["police_report_available"])), \
        "an error occurs in police_report_available range."
    assert set(input_data.auto_make).issubset(set(params["auto_make"])), \
        "an error occurs in auto_make range."
    assert set(input_data.auto_model).issubset(set(params["auto_model"])), \
        "an error occurs in auto_model range."

    assert input_data.months_as_customer.between(params["months_as_customer"][0], params["months_as_customer"][1]).sum() == \
            len(input_data), "an error occurs in months_as_customer range."
    assert input_data.age.between(params["age"][0], params["age"][1]).sum() == \
            len(input_data), "an error occurs in age range."
    assert input_data.policy_number.between(params["policy_number"][0], params["policy_number"][1]).sum() == \
            len(input_data), "an error occurs in policy_number range."
    assert input_data.policy_annual_premium.between(params["policy_annual_premium"][0], params["policy_annual_premium"][1]).sum() == \
            len(input_data), "an error occurs in policy_annual_premium range."
    assert input_data.insured_zip.between(params["insured_zip"][0], params["insured_zip"][1]).sum() == \
            len(input_data), "an error occurs in insured_zip range."
    assert input_data["capital_gains"].between(params["capital_gains"][0], params["capital_gains"][1]).sum() == \
            len(input_data), "an error occurs in capital-gains range."
    assert input_data["capital_loss"].between(params["capital_loss"][0], params["capital_loss"][1]).sum() == \
            len(input_data), "an error occurs in capital-loss range."
    assert input_data.incident_hour_of_the_day.between(params["incident_hour_of_the_day"][0], params["incident_hour_of_the_day"][1]).sum() == \
            len(input_data), "an error occurs in incident_hour_of_the_day range."
    assert input_data.total_claim_amount.between(params["total_claim_amount"][0], params["total_claim_amount"][1]).sum() == \
            len(input_data), "an error occurs in total_claim_amount range."
    assert input_data.injury_claim.between(params["injury_claim"][0], params["injury_claim"][1]).sum() == \
            len(input_data), "an error occurs in injury_claim range."
    assert input_data.property_claim.between(params["property_claim"][0], params["property_claim"][1]).sum() == \
            len(input_data), "an error occurs in property_claim range."
    assert input_data.vehicle_claim.between(params["vehicle_claim"][0], params["vehicle_claim"][1]).sum() == \
            len(input_data), "an error occurs in vehicle_claim range."

In [40]:
check_data(raw_dataset, config_data)

In [41]:
## Save final raw dataset
util.pickle_dump(raw_dataset, config_data["cleaned_raw_dataset_path"])

# <a id='toc7_'></a>[<b>Splitting Data</b>](#toc0_)

In [42]:
# Split input/variable/feature with target/labet/output
X = raw_dataset[config_data["predictor"]].copy()
y = raw_dataset[config_data["label"]].copy()

In [43]:
y.value_counts(normalize=True)

N    0.753
Y    0.247
Name: fraud_reported, dtype: float64

    Data imbalance with 75% and 25%

## <a id='toc7_1_'></a>[<b>Split Train and Test Data (7:3)</b>](#toc0_)

In [44]:
# First split, splitting train and test set with ratio 0.7:0.3 and do stratify splitting
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 42, stratify= y)

## <a id='toc7_2_'></a>[<b>Split Valid and Test Data (1:1)</b>](#toc0_)

In [45]:
# Second split, splitting test and valid set with ratio 0.5:0.5 and do stratify splitting
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state= 42, stratify= y_test)

In [46]:
util.pickle_dump(x_train, config_data["train_set_path"][0])
util.pickle_dump(y_train, config_data["train_set_path"][1])

util.pickle_dump(x_valid, config_data["valid_set_path"][0])
util.pickle_dump(y_valid, config_data["valid_set_path"][1])

util.pickle_dump(x_test, config_data["test_set_path"][0])
util.pickle_dump(y_test, config_data["test_set_path"][1])